# What is it?

A vector search index in Databricks is a structure that stores numeric embeddings for your text data in your delta table

Enables you to quickly search and retrieve the rows that are most similar in meaning to a query.

# Pre-Requisites
1. Vector Search Endpoint
2. Delta Table

## Create Vector Search Endpoint
Vector Search Endpoints are created in Databricks UI in the Compute section.

## 🧱 Create table

In [0]:
CREATE OR REPLACE TABLE workspace.default.demo_product_texts (
  id   STRING,
  name STRING,
  text STRING
);

Insert sample data (10 rows total)

In [0]:
INSERT INTO workspace.default.demo_product_texts (id, name, text)
VALUES
('1', 'AirFlow Runner', 'Lightweight running shoes with breathable mesh'),
('2', 'Mountain Grip', 'Trail running shoes designed for rough terrain'),
('3', 'UrbanEase Casuals', 'Casual sneakers for everyday wear'),
('4', 'SkyGuard Pro', 'High-performance basketball shoes with ankle support'),
('5', 'ComfortWalk Pro', 'Comfortable walking shoes with memory foam insoles'),
('6', 'StormTrek Hiker', 'Waterproof hiking boots for all weather conditions'),
('7', 'LeatherEase Loafers', 'Slip-on loafers with premium leather finish'),
('8', 'FlexCore Trainer', 'Cross-training shoes suitable for gym workouts'),
('9', 'BareMotion Lite', 'Minimalist barefoot shoes for natural running feel'),
('10', 'StreetHigh Classic', 'Fashionable high-top sneakers with durable soles');

In [0]:
Select * from workspace.default.demo_product_texts

Pre-Requisite to Creating Vector search Index: Change Data Feed Enabled

In [0]:
ALTER TABLE workspace.default.demo_product_texts
SET TBLPROPERTIES (
  delta.enableChangeDataFeed = true
);

# Create Vector Search Index
Vector Search Indexes are created in Databricks UI at the table.

#🔍 Query Vector Search Index

#### Example 1:
You want to do a similarity (ANN) search based on a text query.

In [0]:
SELECT *
FROM vector_search(
  index => "workspace.default.demo_product_texts_vsi",   -- fully qualified index name
  query_text => "shoes for gym training",                -- your natural‐language query
  num_results => 5                                      -- how many top results you want
);

#### Example 2: Hybrid search example (semantic + keyword):

In [0]:
SELECT *
FROM vector_search(
  index => "workspace.default.demo_product_texts_vsi",
  query_text => "running shoes with memory foam",
  query_type => "HYBRID",
  num_results => 5
);


#### Example 3: Using Pyspark

In [0]:
%python
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
%python
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk import WorkspaceClient

# Initialize client
w = WorkspaceClient()
vsc = VectorSearchClient(disable_notice=True)

# Get the index
index = vsc.get_index(index_name="workspace.default.demo_product_texts_vsi")

# Run a similarity search using text
results = index.similarity_search(
    query_text = "comfortable shoes for walking long distances",
    columns = ["id","name", "text"],   # only return these columns
    num_results = 5,
    disable_notice=True
)

# Print or show results
for r in results['result']['data_array']:
    print(r)
